In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work\\notebooks'

In [3]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [4]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'heat']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,heat
date,
2010-12-01 00:00:00,432.940036
2010-12-01 01:00:00,473.935901
2010-12-01 02:00:00,483.278761
2010-12-01 03:00:00,545.921252
2010-12-01 04:00:00,550.526112


In [14]:
x = df1['heat'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['heat'] = x_scaled

In [15]:
df1.head()

,heat
date,
2010-12-01 00:00:00,0.537153
2010-12-01 01:00:00,0.667290
2010-12-01 02:00:00,0.696948
2010-12-01 03:00:00,0.895801
2010-12-01 04:00:00,0.910419


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,heat
date,
2010-12-01,0.073040
2010-12-02,-0.212695
2010-12-03,0.291141
2010-12-04,1.031632
2010-12-05,1.426792


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01*3)

In [19]:
len(dftest)

88

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2265

In [22]:
len(dfvali)

567

In [23]:
# best
random.seed(0)

In [24]:
'''
pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20, n_fits=100))
])
'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False, n_fits=100))
])

pipeline.fit(dftrain)

 ARIMA(0,1,0)(0,1,0)[7]             : AIC=9302.233, Time=0.05 sec
 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.49 sec
 ARIMA(0,1,0)(0,1,2)[7]             : AIC=inf, Time=1.58 sec
 ARIMA(0,1,0)(1,1,0)[7]             : AIC=8688.452, Time=0.11 sec
 ARIMA(0,1,0)(1,1,1)[7]             : AIC=inf, Time=0.91 sec
 ARIMA(0,1,0)(1,1,2)[7]             : AIC=inf, Time=2.47 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=8414.378, Time=0.19 sec
 ARIMA(0,1,0)(2,1,1)[7]             : AIC=inf, Time=2.23 sec
 ARIMA(0,1,0)(2,1,2)[7]             : AIC=inf, Time=3.43 sec
 ARIMA(0,1,1)(0,1,0)[7]             : AIC=inf, Time=0.40 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=1.28 sec
 ARIMA(0,1,1)(0,1,2)[7]             : AIC=inf, Time=2.31 sec
 ARIMA(0,1,1)(1,1,0)[7]             : AIC=inf, Time=1.18 sec
 ARIMA(0,1,1)(1,1,1)[7]             : AIC=inf, Time=1.41 sec
 ARIMA(0,1,1)(1,1,2)[7]             : AIC=inf, Time=3.13 sec
 ARIMA(0,1,1)(2,1,0)[7]             : AIC=inf, Time=2.12 sec
 ARIMA(0,

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='ignore', m=7, max_p=3,
                           max_q=3, n_fits=100, stepwise=False, test='adf',
                           trace=True))])

In [ ]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [ ]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False, alpha=0.05, inverse_transform=True)
Actuals = pd.Series.to_numpy(dftrain['heat'])
Residuals = Actuals-Sample_predictions

In [ ]:
Residuals

In [25]:
forecast, conf = pipeline.predict(n_periods=567, return_conf_int=True)
forecast

array([ -0.78180278,   0.16354766,  -0.5894225 ,  -1.00939804,
        -0.81723928,  -0.59460586,  -0.7800789 ,  -1.01235357,
         0.31855071,  -0.54034674,  -1.16629294,  -1.05074485,
        -0.71257443,  -0.79256897,  -1.10388273,   0.67611765,
        -0.47269984,  -1.2673957 ,  -1.11384122,  -0.65844377,
        -1.01999367,  -1.28494292,   0.06579133,  -0.85502407,
        -1.46741211,  -1.3126505 ,  -0.9744221 ,  -1.18619692,
        -1.45460993,   0.02535894,  -0.94643309,  -1.62126351,
        -1.48036598,  -1.10502148,  -1.31896299,  -1.60102664,
        -0.05887303,  -1.07536636,  -1.77161334,  -1.6219704 ,
        -1.23086901,  -1.49486891,  -1.76672239,  -0.31008461,
        -1.27951763,  -1.94014749,  -1.79151667,  -1.42358617,
        -1.65376615,  -1.92767295,  -0.43591035,  -1.42106999,
        -2.0978705 ,  -1.95157746,  -1.57375421,  -1.80907082,
        -2.08513101,  -0.5871776 ,  -1.57812469,  -2.2564853 ,
        -2.10829968,  -1.72904712,  -1.97259649,  -2.24

In [26]:
Actuals = pd.Series.to_numpy(dfvali['heat'])
Actuals

array([-8.19567512e-01, -8.21611364e-01,  2.91991109e+00, -8.37174302e-01,
       -8.25265804e-01,  6.43514153e-01, -7.55491035e-01,  2.31412935e+00,
        2.44258327e+00, -4.51853919e-01, -5.99789911e-02, -9.94356842e-02,
        6.56612299e-01,  1.43012113e+00, -5.04775466e-01, -4.43014036e-01,
        2.76377547e-01, -8.34298577e-01, -3.60211084e-01, -4.70081570e-01,
        5.51095702e-01, -4.15353133e-01, -8.37174302e-01, -2.81534418e-01,
       -4.54825711e-01,  4.79224652e-01,  1.81429597e+00,  1.85126249e+00,
       -1.30122523e-01,  1.52773848e+00,  1.09329802e-02, -4.78522750e-01,
       -4.79713922e-01, -4.36184020e-02, -1.62342766e-02, -4.78549821e-01,
       -1.10383362e-01,  8.54585694e-01,  2.96706165e+00,  1.39038206e+00,
       -8.36615287e-01, -7.03797824e-01, -1.39242873e-01, -3.15765254e-01,
       -5.39037520e-01,  1.35254373e+00, -8.37174302e-01, -6.21374655e-01,
       -8.37174302e-01,  1.95352506e-01,  1.19503190e-01, -5.25937063e-01,
       -5.05936916e-01, -

In [ ]:
forecasttrain, _ = pipeline.predict(n_periods=2265, return_conf_int=True)
forecasttrain

In [ ]:
Actualstrain = pd.Series.to_numpy(dftrain['heat'])
Actualstrain

In [ ]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

In [ ]:
forecasttest, conf = pipeline.predict(n_periods=88, return_conf_int=True)
forecasttest

In [ ]:
Actualstest = pd.Series.to_numpy(dftest['heat'])
Actualstest

In [ ]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

In [ ]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

In [ ]:
mse = mean_squared_error(Actuals, forecast)
mse

In [ ]:
rmse = math.sqrt(mse)
print('RMSE: %f' % rmse)